In [5]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

import pickle

In [6]:
df = pd.read_csv('train_examen.csv')
df.columns

Index([u'checking_status', u'duration', u'credit_history', u'purpose',
       u'credit_amount', u'savings_status', u'employment',
       u'installment_commitment', u'personal_status', u'other_parties',
       u'residence_since', u'property_magnitude', u'age',
       u'other_payment_plans', u'housing', u'existing_credits', u'job',
       u'num_dependents', u'own_telephone', u'foreign_worker', u'class'],
      dtype='object')

In [7]:
##Cambiar todo a números 
le = preprocessing.LabelEncoder()
df = df.apply(le.fit_transform)
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,3,1,1,4,0,3,0,1,0,2,...,3,22,0,1,1,3,0,0,1,1
1,1,1,1,6,101,4,3,3,3,2,...,3,48,1,1,1,1,0,1,1,1
2,0,18,3,3,475,2,2,1,2,2,...,1,3,1,1,0,1,0,0,1,1
3,0,4,3,4,420,4,0,0,0,2,...,3,14,1,1,0,3,0,0,1,1
4,3,4,1,4,292,2,0,3,3,2,...,0,29,1,1,2,1,0,1,1,1


In [8]:
var_disc= [u'checking_status', u'credit_history', u'purpose', u'savings_status',
       u'employment', u'personal_status', u'other_parties',
       u'property_magnitude', u'other_payment_plans', u'housing', u'job',
       u'own_telephone', u'foreign_worker']
var_cont = [ u'duration',  u'credit_amount',u'installment_commitment', u'residence_since', u'age',
        u'existing_credits',
       u'num_dependents']
obj = 'class'

In [9]:
df['ID']=1

In [10]:
from math import log
from __future__ import division

def WoEfd(X, var, obj, ID):
    Y = X.copy()
    IV = 0
    a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
    a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
    for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            IV += float((bads-goods)*d)
            Y[var].replace(y,d,inplace = True)
    print a 
    print "IV total de "+ str(IV)
    return Y
def WoEfc(X, var, obj, ID, bins):
    Y = X.copy()
    IV = []
    for i in bins:
        Y[var] = pd.cut(X[var], bins = i)
        a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
        a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
        IV.append(0)
        for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            IV[-1] += float((bads-goods)*d)
        print "IV con "+str(i) + " bins es " + str(IV[-1])
    armax = np.argmax(IV)
    print "Se el maximiza el IV con "+ str(bins[armax]) + " bins"
    Y[var] = pd.cut(X[var], bins = bins[armax])
    a = Y.pivot_table(aggfunc='count',columns=obj,fill_value=0, index=var,values=ID[0]).reset_index()
    a.loc[-1] =["TOTAL", sum(a[0]), sum(a[1])]
    for y in set(Y[var].values):
            goods = int(a[a[var]==y][1])/int(a[a[var]=='TOTAL'][1])
            bads = a[a[var]==y][0]/int(a[a[var]=='TOTAL'][0])
            d = log(bads/goods)
            Y[var].replace(y,d,inplace = True)
    print a 
    return Y

In [11]:
for x in var_disc:
    df = WoEfd(X=df, var=x, obj = [obj], ID =['ID'])

class checking_status    0    1
 0                  0   72  109
 1                  1  100   92
 2                  2    8   37
 3                  3   35  247
-1              TOTAL  215  485
IV total de 0.714191715278
class credit_history    0    1
 0                 0   23   17
 1                 1   36  161
 2                 2   23   42
 3                 3  114  255
 4                 4   19   10
-1             TOTAL  215  485
IV total de 0.295795260004
class purpose    0    1
 0          0   23   46
 1          1    3    6
 2          2   18   26
 3          3   42   85
 4          4   60   85
 5          5    3    6
 6          6   46  162
 7          7    6    6
 8          8    1    7
 9          9   13   56
-1      TOTAL  215  485
IV total de 0.178516130485
class savings_status    0    1
 0                 0   23   44
 1                 1    7   37
 2                 2  155  272
 3                 3    4   27
 4                 4   26  105
-1             TOTAL  215  485
IV to

In [12]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class,ID
0,-1.140529,1,-0.684375,0.465204,0,-1.096032,-0.047773,1,0.244661,0.017278,...,22,0.359256,-0.198773,1,0.007886,0,0.094194,0.03237,1,1
1,0.896892,1,-0.684375,-0.445444,101,-0.582353,-0.262629,3,-0.164957,0.017278,...,48,-0.126695,-0.198773,1,-0.027418,0,-0.133625,0.03237,1,1
2,0.398829,18,0.008446,0.108529,475,0.251134,0.560062,1,-0.187938,0.017278,...,3,-0.126695,-0.198773,0,-0.027418,0,0.094194,0.03237,1,1
3,0.398829,4,0.008446,0.465204,420,-0.582353,-0.047773,0,0.244661,0.017278,...,14,-0.126695,-0.198773,0,0.007886,0,0.094194,0.03237,1,1
4,-1.140529,4,-0.684375,0.465204,292,0.251134,-0.047773,3,-0.164957,0.017278,...,29,-0.126695,-0.198773,2,-0.027418,0,-0.133625,0.03237,1,1


In [13]:
y = df[obj]
X = df[var_disc + var_cont]

In [14]:
Xt,Xv,yt,yv =train_test_split(X,y,train_size=0.7)
for d in [Xt,Xv,yt,yv]:
    print d.shape

(489, 20)
(211, 20)
(489,)
(211,)


In [15]:
def metricas(model,Xt,Xv,yt,yv):
    print roc_auc_score(y_score=model.predict_proba(Xt)[:,1],y_true=yt)
    print roc_auc_score(y_score=model.predict_proba(Xv)[:,1],y_true=yv)
    print accuracy_score(y_pred=model.predict(Xt),y_true=yt)
    print accuracy_score(y_pred=model.predict(Xv),y_true=yv)
    print confusion_matrix(y_pred=model.predict(Xt),y_true=yt)
    print confusion_matrix(y_pred=model.predict(Xv),y_true=yv)

## Arbol de Desición

In [49]:
param_grid = dict(criterion=['gini','entropy'],
                 max_depth=range(1,11),
                 min_samples_split = range(100,210,10)
                 )

In [50]:
%%time 
model = DecisionTreeClassifier()
grid = GridSearchCV(cv=10,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=130, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
{'min_samples_split': 130, 'criterion': 'gini', 'max_depth': 9}
0.740912707616
CPU times: user 12.8 s, sys: 784 ms, total: 13.6 s
Wall time: 44.7 s


In [51]:
model = grid.best_estimator_
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

0.808900323995
0.706048387097
0.754601226994
0.663507109005
[[104  55]
 [ 65 265]]
[[ 34  22]
 [ 49 106]]


## Regresión Logística 

In [52]:
model =LogisticRegression()
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

0.834248141795
0.81797235023
0.78118609407
0.781990521327
[[ 93  66]
 [ 41 289]]
[[ 31  25]
 [ 21 134]]


## Red neural (TARDA MUCHO)

In [16]:
imputer = Imputer(strategy='mean')
scaler = MinMaxScaler()
pca = PCA()
pipe = make_pipeline(imputer,pca,scaler)

In [17]:
pipe.fit(X)

Pipeline(steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1)))])

In [18]:
Xp=pd.DataFrame(pipe.transform(X),columns=['p%d'%x for x in range(1,len(var_cont)+len(var_disc)+1)])
Xp.head()

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20
0,5.051515e-15,0.425582,0.536535,0.629520,0.544937,0.152614,0.495802,0.339897,0.858901,0.309287,0.256358,0.362252,0.410472,0.590144,0.274126,0.208669,0.429729,0.429024,0.301057,0.538482
1,1.541167e-01,0.924723,0.583204,0.241482,0.314234,0.800619,0.501889,0.152548,0.623182,0.713973,0.330048,0.299054,0.380394,0.471767,0.200542,0.506631,0.271240,0.318300,0.683824,0.172040
2,7.251532e-01,0.046358,0.328661,0.697214,0.493873,0.631129,0.311269,0.257713,0.245237,0.333031,0.354972,0.461604,0.578858,0.618626,0.225788,0.613803,0.140433,0.386941,0.203247,0.227787
3,6.408012e-01,0.265183,0.754958,0.530672,0.553370,0.622972,0.281414,0.262029,0.692118,0.291342,0.346065,0.154663,0.325730,0.443840,0.128175,0.432461,0.407172,0.415411,0.280240,0.486387
4,4.455554e-01,0.556707,0.639003,0.155034,0.116815,0.264345,0.683933,0.511891,0.196311,0.303441,0.272194,0.423302,0.357807,0.590578,0.192595,0.596866,0.279598,0.459463,0.607972,0.163174


In [19]:
param_grid= dict(hidden_layer_sizes=[(i,) for i in range(5,21)],
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive'],
max_iter = [10000])

In [60]:
%%time 
model = MLPClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)

grid.fit(Xp,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [ ]:
model = grid.best_estimator_
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

## Máquina Vector Soporte

In [20]:
imputer = Imputer(strategy='mean')
scaler = MinMaxScaler()
pca = PCA()
pipe = make_pipeline(imputer,pca,scaler)
pipe.fit(X)
Xp=pd.DataFrame(pipe.transform(X),columns=['p%d'%x for x in range(1,len(var_cont)+len(var_disc)+1)])

In [21]:
param_grid = dict(kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
degree= range(1,6),
decision_function_shape = ('ovo', 'ovr'))

In [22]:
%%time 
model = SVC()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)

grid.fit(Xp,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=1, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
{'kernel': 'linear', 'decision_function_shape': 'ovo', 'degree': 1}
0.804085158333
CPU times: user 1.42 s, sys: 120 ms, total: 1.54 s
Wall time: 6.37 s


## K-Vecinos

In [70]:
param_grid=dict(algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute'],
n_neighbors = range(50,201))

In [71]:
%%time 
model = KNeighborsClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)

grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=163, p=2,
           weights='uniform')
{'n_neighbors': 163, 'algorithm': 'auto'}
0.607892587851
CPU times: user 16.7 s, sys: 1.05 s, total: 17.8 s
Wall time: 2min 38s


In [73]:
model = grid.best_estimator_
metricas(model,Xt,Xv,yt,yv)

0.581761006289
0.639458525346
0.674846625767
0.734597156398
[[  0 159]
 [  0 330]]
[[  0  56]
 [  0 155]]


## Gradiente Descendiente Estocástico 

In [80]:
param_grid = dict(penalty = ['l2', 'l1','elasticnet'],
alpha = np.arange(0.0001,0.001),
loss = ['log']
#loss = ['hinge', 'log', 'modified_huber','squared_hinge', 
#'perceptron']
                 )

In [81]:
%%time 
model = SGDClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)

grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l1', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
{'penalty': 'l1', 'alpha': 0.0001, 'loss': 'log'}
0.582296043617
CPU times: user 288 ms, sys: 80 ms, total: 368 ms
Wall time: 763 ms


In [82]:
model = grid.best_estimator_
metricas(model,Xt,Xv,yt,yv)

0.528444825615
0.516877880184
0.370143149284
0.298578199052
[[156   3]
 [305  25]]
[[ 55   1]
 [147   8]]


/home/aleespa/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


## Bayes Ingenuo 

In [83]:
model = GaussianNB()
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

0.814408233276
0.783294930876
0.758691206544
0.729857819905
[[110  49]
 [ 69 261]]
[[ 39  17]
 [ 40 115]]


## Ensambles

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [85]:
param_grid = dict(criterion=['gini','entropy'],
                 max_depth=range(1,11),
                 min_samples_split = range(100,210,10)
                 )

In [86]:
%%time 
model = RandomForestClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_
rf = grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=120, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
{'min_samples_split': 120, 'criterion': 'entropy', 'max_depth': 9}
0.79404367446
CPU times: user 13.8 s, sys: 808 ms, total: 14.6 s
Wall time: 1min 59s


In [87]:
param_grid = dict(n_estimators = range(5,30),
learning_rate = np.arange(0.1,1.1,0.1))

In [88]:
%%time 
model = AdaBoostClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_
adab = grid.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.5, n_estimators=19, random_state=None)
{'n_estimators': 19, 'learning_rate': 0.5}
0.776837681692
CPU times: user 11.4 s, sys: 792 ms, total: 12.2 s
Wall time: 2min 48s


In [89]:
vc = VotingClassifier(estimators=[('adab', adab), ('rf', rf)],
                 voting='soft')
vc.fit(Xt,yt)
metricas(vc,Xt,Xv,yt,yv)

0.835153421002
0.783467741935
0.773006134969
0.758293838863
[[ 58 101]
 [ 10 320]]
[[ 12  44]
 [  7 148]]


In [4]:
from xgboost.sklearn import XGBClassifier

OSError: /home/aleespa/anaconda2/lib/python2.7/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.20' not found (required by /home/aleespa/anaconda2/lib/python2.7/site-packages/xgboost/./lib/libxgboost.so)

In [ ]:
xgb = XGBClassifier()
xgb.fit(Xt,yt)
metricas(xgb,Xt,Xv,yt,yv)